# Extraction notebook

This notebook is responsible for the extraction of Sales tables from AdventureWorks database.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install pyodbc pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

ModuleNotFoundError: No module named 'dbutils'

In [7]:
# Recuperando credenciais do Databricks Secret Scope
host = dbutils.secrets.get(scope="mateus_marin_adw", key="host")
database = dbutils.secrets.get(scope="mateus_marin_adw", key="database")
login = dbutils.secrets.get(scope="mateus_marin_adw", key="login")
password = dbutils.secrets.get(scope="mateus_marin_adw", key="password")
port = dbutils.secrets.get(scope="mateus_marin_adw", key="port")


managed_location = dbutils.winget.get("managed_location")

ResourceDoesNotExist: Failed to get secret password for scope mateus_marin_adw.

In [0]:
# Criando a string de conexão
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={host};
    DATABASE={database};
    UID={username};
    PWD={password};
"""

# Criando conexão com o SQL Server
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    print("Conexão com SQL Server estabelecida com sucesso!")
except Exception as e:
    print(f"Erro ao conectar ao SQL Server: {e}")